In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import urllib.request
import os
import wget
import time
from urllib.error import HTTPError

In [ ]:
username = 'USERNAME'

In [ ]:
search_name = 'from:'+username

# Creating list to append tweet data to
tweets_list1 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
# for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:lumiebre').get_items()):
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(search_name).get_items()): #you set the twitter handle name here
    tweets_list1.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.media,tweet.retweetedTweet,tweet.quotedTweet])
#     if i>maxTweets:
#         break
#     tweets_list1.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.media,tweet.retweetedTweet,tweet.quotedTweet])

# Creating a dataframe from the tweets list above
tweets_df1 = pd.DataFrame(tweets_list1, columns=['Datetime', 'Tweet Id', 'Text', 'Username',"Media", "Retweet","Quote"])

# Display first 5 entries from dataframe
tweets_df1.head()

In [ ]:
# checks to see if there is media in a tweet and if so how many items

def photoNum(media):
    if media is None:
        return 0
    else:
        return len(media)

In [ ]:
# adds a new column to the data frame containing how many pieces of media are in a tweet

tweets_df1["Num_Images"] = tweets_df1["Media"].map(photoNum)

In [ ]:
# checks for existing "twitter_backup" and "username" directories in working directory
# and creates them if they do not already exist
try:
    os.mkdir('twitter_backup')
    os.chdir('twitter_backup')
except:
    os.chdir('twitter_backup')
try:
    os.mkdir(username)
    os.chdir(username)
except:
    os.chdir(username)

# saves contents of dataframe to csv  
output_file = username+'.csv'
tweets_df1[['Datetime', 'Tweet Id', 'Num_Images','Text','Retweet', 'Quote']].to_csv(output_file)

In [ ]:
# scrapes images from tweets with photos & thumbnail previews from tweets with videos
# and saves them to a file named with the index number in the csv

# if your laptop goes to sleep or you need to restart after an error
# replacing 0 in the second line with the index number 
# in the filename of the last image downloaded
# i.e. for "tweed_id87photo0.jpg" it's 87

for index, row in tweets_df1.iterrows():
    if row.Media is not None:
        if index > 0:
            for i in range(len(row.Media)):
                img = row.Media[i]
                if isinstance(img, sntwitter.Photo):
                    url = img.fullUrl
                    destination = 'tweet_id{0}photo{1}.jpg'.format(index,i)
                    print(destination)
                else:
                    try:
                        url = img.thumbnailUrl
                        destination = 'tweet_id{0}mediathumbnail{1}.jpg'.format(index,i)
                        wget.download(url, destination)
                        print(destination)
                    except HTTPError as err:
                        print(err.code)
                        print("timeout... waiting 3 mins and trying again")
                        time.sleep(180)
                        continue
                    else:
                        continue